In [1]:
from langchain.schema import Document
import json

# 파일 경로 
file_paths = {
    "term": "./metadata/term.json",
    "load_traffic_law": "./metadata/load_traffic_law.json",
    "modifier": "./metadata/modiflier.json",
    "car_case": "./metadata/car_to_car.json",
    "precedent": "./metadata/precedent.json"
}

# 교통사고 케이스용 필드 상수
CASE_ID = "사건 ID"
CASE_TITLE = "사건 제목"
CASE_SITUATION = "사고상황"
BASE_RATIO = "기본 과실비율"
MODIFIERS = "케이스별 과실비율 조정예시"
LAW_REFERENCES = "관련 법규"
PRECEDENT = "참고 판례"
REASON = "기본 과실비율 해설"

# JSON 로드 함수
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 리스트형 JSON 변환 (term, modifier, law_meta)
def convert_list_to_documents(data_list, doc_type):
    return [
        Document(page_content=json.dumps(item, ensure_ascii=False), metadata={"type": doc_type})
        for item in data_list
    ]

def convert_precedent_documents(data_list):
    return [
        Document(
            page_content=f"{item['court']} {item['case_id']} : {item['content']}",
            metadata={
                "court": item["court"],
                "case_id": item["case_id"],
            }
        ) for item in data_list
    ]

def convert_car_case_documents(data_list):
    documents = []

    def safe_value(value):
        if isinstance(value, list):
            return ", ".join(map(str, value))
        elif isinstance(value, dict):
            return json.dumps(value, ensure_ascii=False)
        elif value is None:
            return ""  # null도 허용 안 되므로 빈 문자열로 처리
        else:
            return str(value)

    for item in data_list:
        if not isinstance(item, dict):
            continue

        # page_content는 원본 전체 JSON 문자열
        content = json.dumps(item, ensure_ascii=False)

        # 기본 과실비율 해설이 리스트일 수 있음 → 문자열로 병합
        reason = item.get(REASON)
        if isinstance(reason, list):
            reason = "\n".join(map(str, reason))

        metadata = {
            "type": "car_case",
            "id": safe_value(item.get(CASE_ID)),
            "title": safe_value(item.get(CASE_TITLE)),
            "situation": safe_value(item.get(CASE_SITUATION)),
            "base_ratio": safe_value(item.get(BASE_RATIO)),
            "modifiers": safe_value(item.get(MODIFIERS)),
            "load_traffic_law": safe_value(item.get(LAW_REFERENCES)),
            "precedent": safe_value(item.get(PRECEDENT)),
            "reason": safe_value(reason)
        }

        documents.append(Document(page_content=content, metadata=metadata))
    return documents

# 도로교통법 law JSON → 문서화
def convert_law_json_to_documents(data_dict):
    documents = []

    def normalize(item):
        return json.dumps(item, ensure_ascii=False) if isinstance(item, dict) else str(item)

    for law_name, content in data_dict.items():
        if isinstance(content, dict):
            for clause, text in content.items():
                lines = [normalize(x) for x in (text if isinstance(text, list) else [text])]
                full_text = f"{law_name} {clause}\n" + "\n".join(lines)
                documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
        else:
            lines = [normalize(x) for x in (content if isinstance(content, list) else [content])]
            full_text = f"{law_name}\n" + "\n".join(lines)
            documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
    
    return documents


# 문서화 실행
term_docs = convert_list_to_documents(load_json(file_paths["term"]), "term")
modifier_docs = convert_list_to_documents(load_json(file_paths["modifier"]), "modifier")
precedent_docs = convert_precedent_documents(load_json(file_paths["precedent"]))
car_case_docs = convert_car_case_documents(load_json(file_paths["car_case"]))
load_traffic_law_docs = convert_law_json_to_documents(load_json(file_paths["load_traffic_law"]))


# 전체 문서 리스트
#all_docs = term_docs + modifier_docs + car_case_docs + precedent_docs + load_traffic_law_docs
all_docs = precedent_docs

In [2]:
from langchain.vectorstores import Chroma, Annoy, FAISS # persist 지원
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings


embedding_model = HuggingFaceEmbeddings(model_name="upskyy/bge-m3-korean", encode_kwargs={'normalize_embeddings': True})

vectorstore = FAISS.from_documents(
    documents=all_docs,
    embedding=embedding_model
    # metric='angular',
    # n_trees=10,
    # persist_directory="./vectordb"
)
# vectorstore = 
# vectorstore.persist()  

c:\Users\Playdata\anaconda3\envs\vectordb_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# 1. LLM 설정
model = ChatOpenAI(model='gpt-4o', temperature=0.5)

# 2. 프롬프트 템플릿
prompt = PromptTemplate(
    template="""
아래 문서 내용을 바탕으로 사용자가 물어본 용어나 법률 조항, 판례에 대해 정확하고 간결하게 설명해 주세요.

질문: {question}

문서: {context}

답변 형식:
- 용어/조항 정의: [정확한 설명]
- 출처가 명시된 경우: 관련 법률/조문 번호/판례명을 반드시 포함

답변:
""",
    input_variables=["question", "context"]
)

# 3. 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 4. QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)


In [4]:
# 5. 실제 질의 실행
query = "대법원90도1116에 대해 알려줘"

res = qa_chain.invoke({"query": query})

# 6. 출력
print("✅ 답변:\n", res["result"])

✅ 답변:
 - 판례 설명: 대법원 90도1116 판결은 횡단보도의 표지판이나 신호등이 설치되어 있지 않지만, 도로 바닥에 페인트로 횡단보도 표시가 되어 있는 곳에서 발생한 교통사고에 관한 것입니다. 이 판결에서는 피고인이 운전하는 차로 쪽에 횡단보도 표시가 식별 가능할 정도로 남아 있었기 때문에, 해당 사고가 도로교통법상 횡단보도에서 일어난 것으로 인정되었습니다. 이 판결은 횡단보도의 표지가 명확하지 않더라도 도로 표시에 의해 횡단보도로 인정될 수 있음을 보여줍니다.
